# Notebook demo of using vqe module to solve a QUBO

In [1]:
import sys
sys.path.append('../')

from aquapointer.digital.loaddata import LoadData
from aquapointer.digital.qubo import Qubo
import aquapointer.digital.qubo_utils as qutils
from aquapointer.digital.vqe import VQE

# General imports
import numpy as np

# Pre-defined ansatz circuit, operator class
from qiskit.circuit.library import QAOAAnsatz
from qiskit.primitives import Sampler
from qiskit import transpile 
from qiskit import Aer

backend = Aer.get_backend('aer_simulator')
sampler = Sampler(options={"shots": int(1e4)})

In [2]:
# LoadData does all the file loading
ld = LoadData()

# Qubo computes all the qubo matrices given the 3d rism files and rescaled positions of registers from LoadData.
q = Qubo(ld)

In [3]:
# use the first slice as an example to solve
qubo, ising_ham = q.qubo_hamiltonian_pairs[0]
num_qubits = len(qubo)

# QAOA ansatz circuit
qaoa_ansatz = QAOAAnsatz(ising_ham, reps=1)

from qiskit.circuit.library import TwoLocal
# TwoLocal ansatz circuit
twolocal_ansatz = TwoLocal(num_qubits, 'ry', 'cx',  entanglement='linear', reps=1)
qubo

array([[-0.01074377,  0.04727714,  0.04727714,  0.04727714],
       [ 0.04727714, -0.00892812,  0.03218907,  0.04727714],
       [ 0.04727714,  0.03218907, -0.00865155,  0.04727714],
       [ 0.04727714,  0.04727714,  0.04727714, -0.00969204]])

In [4]:
# classical brute-force solution
sol, ref_value = q.find_optimum(qubo=qubo)
sol, ref_value

('1000', -0.010743767178636635)

In [5]:
def prob_optimal_solution(vqe_object: VQE, alpha: float, maxiter: int, fraction: float, verbose=False) -> tuple[float, int]:

    res = vqe_object.run(alpha=alpha, maxiter=maxiter, method='COBYLA')
    nfev = res.nfev

    # Assign solution parameters to ansatz
    qc = vqe_object.ansatz.assign_parameters(vqe_object.params)
    # Add measurements to our circuit
    qc.measure_all()
    # Sample ansatz at optimal parameters
    samp_dist = sampler.run(qc, shots=int(1e4)).result().quasi_dists[0]
    samp_dist_binary = samp_dist.binary_probabilities()

    correct_dist = {}
    for key in samp_dist_binary.keys():
        reverse_key = key[::-1]
        keynot = [(int(b)+1)%2 for b in reverse_key]
        correct_dist[''.join(map(str, keynot))] = samp_dist_binary[key]

    prob_energy = []
    bitstrings = []
    for key in correct_dist.keys():
        key_np = np.fromiter(map(int, key), dtype=int)
        prob_energy.append([correct_dist[key], qutils.ising_energy(key_np, qubo)])
        bitstrings.append(key)

    bitstrings = np.array(bitstrings)
    prob_energy = np.array(prob_energy)

    sorted_indices = np.argsort(prob_energy[:, 1])
    sorted_keys = bitstrings[sorted_indices]
    sorted_values = prob_energy[:, 1][sorted_indices]

    opt_energy = sorted_values[0]
    opt_b = sorted_keys[0]
    
    #pick top 10% of lowest observed energies and compute probability mass on them
    n = int(len(prob_energy)*fraction)
    if verbose:
        if n<=1:
            print(prob_energy)

    total_mass = 0.0
    top_avg_energy = 0.0
    for i in range(n):
        total_mass += correct_dist[sorted_keys[i]]
        top_avg_energy += sorted_values[i]*correct_dist[sorted_keys[i]]
        if verbose:
            print(sorted_values[i], correct_dist[sorted_keys[i]])
    top_avg_energy = top_avg_energy / total_mass
    return round(top_avg_energy, 5), round(total_mass, 3), nfev, (opt_b, opt_energy)  

In [6]:
# run the optimization for a given vqe_object and a list of confidence intervals
def run_optimization(vqe_object, alphas, maxiter):
    nfevs = [] #this is for book keeping the number of function evaluations    
    opt_energy = np.inf

    for alpha in alphas:
        top_avg_energy, total_mass, nfev, opt = prob_optimal_solution(vqe_object, alpha, maxiter, 0.2, verbose=False)
        nfevs.append(nfev)
        if opt_energy>opt[1]:
            opt_energy = opt[1]
            res = opt
            print(opt)
        print(f"{alpha}, {top_avg_energy}, {total_mass}")
    return nfevs, res

#helper function to get number of basis gates used in ansatz
def total_gates(ansatz):
    basis_gates=['u1', 'u2', 'u3', 'cx']

    ansatz_transpiled = transpile(qaoa_ansatz, backend, basis_gates=['u1', 'u2', 'u3', 'cx'], optimization_level=2)
    gates = ansatz_transpiled.count_ops()

    total_gates = 0
    for gate in basis_gates:
        total_gates += gates.get(gate, 0)
    return total_gates

In [7]:
# EXAMPLE: QAQA

alphas = [1.0, 0.75, 0.5, 0.25, 0.1]
beta  = [0.7977]#, 0.7905, 0.5657]#, 0.4189]#, 0.3575, 0.3279, 0.2785, 0.1911, 0.1384, 0.0885]
gamma = [0.0765]#, 0.1634, 0.3662]#, 0.5890]#, 0.7046, 0.7594, 0.8345, 0.9352, 0.9529, 0.9976]
params = beta+gamma
vqe_qaoa = VQE(qubo=qubo, ansatz=qaoa_ansatz, ising_ham=ising_ham, sampler=sampler, params=params)

nfevs_qaoa = run_optimization(vqe_qaoa, alphas, maxiter=20)

('1000', -0.010743767178636635)
1.0, -0.00978, 0.189
0.75, -0.00979, 0.248
0.5, -0.00978, 0.308
0.25, -0.0098, 0.321
0.1, -0.00976, 0.338


In [8]:
#EXAMPLE: linear entanglement ansatz

vqe_linear = VQE(qubo=qubo, ansatz=twolocal_ansatz, ising_ham=ising_ham, sampler=sampler, params=None)
alphas = [1.0, 0.75, 0.5, 0.25, 0.1]

nfevs_linear = run_optimization(vqe_linear, alphas, maxiter=60)

('1000', -0.010743767178636635)
1.0, -0.01072, 0.167
0.75, -0.00969, 0.754
0.5, -0.00988, 0.506
0.25, -0.01074, 0.334
0.1, -0.01074, 0.179


In [9]:
# now we solve all the qubo's and output optimal bitstrings of for each slice using QAOA and twolocal ansatz
alphas = [1.0, 0.66, 0.33]
nfevs_qaoa_slices = []
nfevs_linear_slices = []

res_qaoa_slices = []
res_linear_slices = []

for qubo, ising_ham in q.qubo_hamiltonian_pairs:
    
    num_qubits = len(qubo)
    qaoa_ansatz = QAOAAnsatz(ising_ham, reps=1)
    beta  = [0.7977]#, 0.7905, 0.5657]#, 0.4189]#, 0.3575, 0.3279, 0.2785, 0.1911, 0.1384, 0.0885]
    gamma = [0.0765]#, 0.1634, 0.3662]#, 0.5890]#, 0.7046, 0.7594, 0.8345, 0.9352, 0.9529, 0.9976]
    params = beta+gamma
    vqe_qaoa = VQE(qubo=qubo, ansatz=qaoa_ansatz, ising_ham=ising_ham, sampler=sampler, params=params)

    #using linear entanglement ansatz
    # TwoLocal ansatz circuit
    twolocal_ansatz = TwoLocal(num_qubits, 'ry', 'cx',  entanglement='linear', reps=1)
    vqe_linear = VQE(qubo=qubo, ansatz=twolocal_ansatz, ising_ham=ising_ham, sampler=sampler, params=None)

    # compute number of gates to specify maxiter for the different ansatzes
    # to keep it fair, we want number_of_gates * maxiter to be the same for both instances
    gates_qaoa, gates_twolocal = total_gates(qaoa_ansatz), total_gates(twolocal_ansatz)
    maxiter_qaoa = 20
    maxiter_linear = maxiter_qaoa * (gates_qaoa//gates_twolocal)

    print('qaoa')
    nfevs_qaoa, res_qaoa = run_optimization(vqe_qaoa, alphas, maxiter=maxiter_qaoa)
    nfevs_qaoa_slices.append(nfevs_qaoa)
    res_qaoa_slices.append(res_qaoa)

    print('linear')
    nfevs_linear, res_linear = run_optimization(vqe_linear, alphas, maxiter=maxiter_linear)
    nfevs_linear_slices.append(nfevs_linear)
    res_linear_slices.append(res_linear)


qaoa
('1000', -0.010743767178636635)
1.0, -0.00979, 0.191
0.66, -0.00976, 0.247
0.33, -0.00979, 0.347
linear
('1000', -0.010743767178636635)
1.0, -0.01058, 0.123
0.66, -0.00975, 0.112
0.33, -0.01047, 0.444
qaoa
('010000001', -0.05676551209085727)
1.0, 0.02869, 0.191
0.66, -0.00436, 0.723
0.33, -0.01434, 0.842
linear
('010000001', -0.05676551209085727)
1.0, 0.00383, 0.696
0.66, -0.02119, 0.754
0.33, -0.02385, 0.794
qaoa
('10001000000001', -0.07861404665042948)
1.0, 0.04144, 0.25
0.66, 0.00224, 0.366
0.33, -0.00099, 0.381
linear
('00000001010000', -0.07361176795056312)
1.0, 0.04984, 0.393
('00001001000000', -0.07443814919873588)
0.66, -0.01885, 0.531
0.33, -0.0183, 0.51
qaoa
('01000000101000', -0.06639706847918819)
1.0, 0.17129, 0.199
('10000000100100', -0.07111143020268126)
0.66, -0.01221, 0.553
0.33, -0.01926, 0.711
linear
('10000000000011', -0.06611651718391577)
1.0, -0.03327, 0.186
0.66, 0.00657, 0.402
0.33, -0.02958, 0.407
qaoa
('0000000010010', -0.05191488530963839)
1.0, 0.19177, 0

In [10]:
res_qaoa_slices

[('1000', -0.010743767178636635),
 ('010000001', -0.05676551209085727),
 ('10001000000001', -0.07861404665042948),
 ('10000000100100', -0.07111143020268126),
 ('0000010100000', -0.05195638819992217),
 ('00010000', -0.02404680978135616)]

In [11]:
res_linear_slices

[('1000', -0.010743767178636635),
 ('010000001', -0.05676551209085727),
 ('00001001000000', -0.07443814919873588),
 ('10000000000011', -0.06611651718391577),
 ('0000010100000', -0.05195638819992217),
 ('00010000', -0.02404680978135616)]

In [ ]:
# now apply the function that maps these bitstrings to water molecule position

# Resource estimation

In [ ]:
# number_of_gates * maxiter is the same for both approaches, this makes a fair comparison in performance